In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [32]:
lstm = nn.LSTM(3,3)
inputs = [torch.randn(1,3) for _ in range(5)]

hidden = (torch.randn(1,1,3),
         torch.randn(1,1,3))

for i in inputs:
    out, hidden = lstm(i.view(1,1,-1),hidden)
    
inputs = torch.cat(inputs).view(len(inputs),1,-1)
hidden = (torch.randn(1,1,3),torch.randn(1,1,3))
out, hidden = lstm(inputs,hidden)
print(out)
print(hidden)    

tensor([[[-0.0762, -0.3406,  0.2993]],

        [[-0.1676, -0.2791,  0.3190]],

        [[ 0.0451, -0.2200,  0.2012]],

        [[ 0.1141, -0.3164,  0.0424]],

        [[-0.2084, -0.0968,  0.2488]]], grad_fn=<StackBackward>)
(tensor([[[-0.2084, -0.0968,  0.2488]]], grad_fn=<StackBackward>), tensor([[[-0.3156, -0.3536,  0.5693]]], grad_fn=<StackBackward>))


In [33]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype = torch.long)

training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]

word_to_ix = {}
for sent, tags in training_data:
        for word in sent:
            if word not in word_to_ix:
                word_to_ix[word] = len(word_to_ix)

print(word_to_ix)
tag_to_ix = {"DET" : 0, "NN" : 1, "V" : 2}

EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [41]:
class LSTMTagger(nn.Module):
    
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
    
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence),-1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [42]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(),lr=0.1)

with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)
    
for epoch in range(300):
    for sentence, tags in training_data:
        model.zero_grad()
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        
        tag_scores = model(sentence_in)
        
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0],word_to_ix)
    tag_scores = model(inputs)
    
    print(tag_scores)

tensor([[-1.1008, -1.1663, -1.0331],
        [-1.0508, -1.2759, -0.9911],
        [-1.0359, -1.2858, -0.9979],
        [-1.0199, -1.3122, -0.9938],
        [-1.1498, -1.1347, -1.0167]])
tensor([[-0.1283, -2.1820, -4.8832],
        [-3.6760, -0.0660, -3.2570],
        [-4.0942, -3.4976, -0.0481],
        [-0.0331, -3.9648, -4.3015],
        [-4.8669, -0.0210, -4.3378]])
